In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import mnist

In [2]:
def normalize_img(image, label):
    return (tf.cast(image, tf.float32) / 255.0, label)

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

train_dataset = train_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Store dataset in cache
train_dataset = train_dataset.cache()
# Shuffle data
train_dataset = train_dataset.shuffle(len(train_dataset))
# Split up into a batch of 64
train_dataset = train_dataset.batch(128)
# Grab other stuff before it done dealing with current stuff
# Optimization 
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)



In [4]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

test_dataset = test_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Split up into a batch of 64
test_dataset = test_dataset.batch(128)
# Store dataset in cache
test_dataset = test_dataset.cache()
# Grab other stuff before it done dealing with current stuff
# Optimization 
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [5]:
# Visualise image tensor (batch has to be 1)
visualise = False
if visualise: 
    for (img, label) in train_dataset:
        print(img.numpy(), label.numpy())
        break
    else:
        pass

In [6]:
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
# model.add(tf.keras.layers.Dense(128, activation = 'relu'))
# model.add(tf.keras.layers.Dense(128, activation = 'relu'))
# model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_dataset,
#                     epochs=50)

# model.save('handwritten.model')
#model.summary()

In [7]:
model = tf.keras.models.load_model('handwritten.model')

loss, accuracy = model.evaluate(train_dataset)

print(loss)
print(accuracy)

2022-09-29 05:18:43.240641: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 1s 665us/step - loss: 0.0050 - accuracy: 0.9986
0.004973487462848425
0.9986000061035156


In [8]:
# from matplotlib.lines import lineStyles
# import matplotlib.pyplot as plt
# plt.figure(figsize=(18,8))
# plt.plot(0,0, history.history['accuracy'], label='Accuracy', lw=3)
# plt.plot(0,0, history.history['loss'], label='Loss', lw=3, linestyle='--')
# plt.legend


In [9]:
def rescaleFrame(frame, scale):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[1] * scale)
    print(width, height)
    dimensions = (width, height)
    
    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

In [23]:
capture = cv.VideoCapture(0)
scale = 0.50

while True:
    isTrue, frame = capture.read()
    frame_resized = rescaleFrame(frame, scale)

    # Grayscaled
    gray = cv.cvtColor(frame_resized, cv.COLOR_BGR2GRAY)
    gray_28 = cv.resize(gray, (28,28), interpolation=cv.INTER_AREA)
    
    gray_28 = gray_28.astype("float32") / 255.0
    gray_28 = np.expand_dims(gray_28, axis=-1)
    
    prediction = model.predict(gray_28.reshape(1,28,28))
    predargmax = np.argmax(prediction)

    # Black and white
    range = (150, 255)
    #range = (125, 255)
    ret, thresh = cv.threshold(gray, range[0], range[1], cv.THRESH_BINARY)
    contours, hierarchies = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    # 28
    ret_28, thresh_28 = cv.threshold(gray, range[0], range[1], cv.THRESH_BINARY)
    thresh_28 = cv.resize(thresh_28, (28,28), interpolation=cv.INTER_AREA)
    contours_28, hierarchies_28 = cv.findContours(thresh_28, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    thresh_28 = thresh_28.astype("float32") / 255.0
    thresh_28 = np.expand_dims(thresh_28, axis=-1)
    
    
    prediction_thresh_28 = model.predict(thresh_28.reshape(1,28,28))
    predargmax_thresh_28 = np.argmax(prediction_thresh_28)


    cv.putText(gray, 'Gray: ' + str(predargmax), (20,40), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv.putText(gray, 'B&W: ' + str(predargmax_thresh_28), (20,80), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
    
    cv.imshow('Gray scale', gray)
    cv.imshow('Black & White (gray threshold)', thresh)
    cv.imshow('Black & White (gray threshold) 28x28', thresh_28)
    
    cv.moveWindow('Black & White (gray threshold)', cv.getWindowImageRect('Gray scale')[2], 0)
    cv.moveWindow('Black & White (gray threshold) 28x28', cv.getWindowImageRect('Gray scale')[2], 0)
   

    if cv.waitKey(100) & 0xFF==ord('q'):
        break
    
capture.release()
cv.waitKey(100)
cv.destroyAllWindows()
cv.waitKey(100)

640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 10ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/s

-1

: 